# 1 Data Preparation and Loading

In [1]:
# Go to project root folder
import os
os.chdir("../")
%pwd

'/workspaces/car_vs_ambulance_classification_ensemble_cnn'

In [2]:
# automatically reload all modules before executing
%load_ext autoreload
%autoreload 2 

In [3]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.config.list_physical_devices('GPU'), tf.__version__

2024-09-28 14:55:59.960553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-28 14:55:59.971979: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-28 14:55:59.975787: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-28 14:55:59.985279: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1727535361.356284   12991 cuda_executor.c

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.17.0')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import utils
from src.utils.image_data_loader import ImageDataLoader
import opendatasets as od
import random
import pathlib
import os
import glob
import cv2

In [5]:
PARAMS_YAML = 'params.yaml'
params_config = utils.utils.read_yaml(yaml_path=PARAMS_YAML)
params_config

2024-09-28 14:56:01,783 — utils.utils — INFO — yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully


ConfigBox({'base': {'random_seed': 42}, 'data_split': {'test_size': 0.2}, 'train': {'image_size': 224, 'batch_size': 32, 'binary_crossentropy': 'binary_crossentropy', 'fine_tune_args': None, 'letnet': {'base_lr': 0.01, 'epochs': 2}, 'mini_vgg_net': {'base_lr': 0.01, 'epochs': 10}, 'shallownet': {'base_lr': 0.01, 'epochs': 10}}, 'evaluate': {'metrics': ['accuracy', 'precision', 'recall'], 'reports_dir': 'reports', 'metrics_file': 'metrics.json', 'confusion_matrix_image': 'confusion_matrix.png', 'confusion_matrix_data': 'confusion_matrix_data.csv'}, 'paths': {'dataset_url': 'https://www.kaggle.com/datasets/amansingh0000000/image-classification-and-yolo-dataset/data', 'raw_dataset_dir': 'dataset/raw', 'dataset_path': 'dataset/image-classification-and-yolo-dataset/Ambulance_Car', 'preprocess_dataset': 'dataset/preprocess', 'train_dir': 'dataset/raw/image-classification-and-yolo-dataset/Ambulance_Car/Train', 'labels_dir': 'dataset/raw/image-classification-and-yolo-dataset/Ambulance_Car/Labe

In [6]:
tf.random.set_seed(params_config.base.random_seed)

In [7]:
image_loader = ImageDataLoader(params_yaml=PARAMS_YAML)
train_ds, val_ds = image_loader.load_image_dataset()

2024-09-28 14:56:01,840 — utils.utils — INFO — yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully


Found 2227 files belonging to 2 classes.


I0000 00:00:1727535361.881493   12991 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727535361.883481   12991 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727535361.885256   12991 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727535361.993938   12991 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Found 557 files belonging to 2 classes.
2024-09-28 14:56:02,743 — src.utils.image_data_loader — INFO — load_image_dataset completed


# 2 Model Training

In [21]:
from models import lenet, mini_vgg_net, shallownet
from tensorflow.keras.optimizers import SGD

In [22]:
lenet_model = lenet.LeNet(params_yaml=PARAMS_YAML)
lenet_model.build()
lenet_model.get_summary()

2024-09-28 15:06:26,924 — utils.utils — INFO — yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully
2024-09-28 15:06:26,959 — models.lenet — INFO — LeNet model built successfully.


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 224, 224, 20)   │         1,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 112, 112, 20)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 112, 112, 20)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 112, 112, 50)   │        25,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 56, 56, 50)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 56, 56, 50)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 156800)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 156800)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 500)            │    78,400,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           501 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,427,571 (299.18 MB)

 Trainable params: 78,427,571 (299.18 MB)

 Non-trainable params: 0 (0.00 B)

### LeNet Train

In [23]:
# train the network
lenet_model_opt = SGD(learning_rate=1e-2,
                      weight_decay=0.01 / 40,
                      momentum=0.9,
                      nesterov=True)
history = lenet_model.train(train_data=train_ds,
                            val_data=val_ds,
                            optimizer=lenet_model_opt)

Epoch 1/2
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.2424 - loss: nan - precision: 0.7830 - recall: 0.1136 - val_accuracy: 0.1382 - val_loss: nan - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/2
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.1495 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.1382 - val_loss: nan - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
2024-09-28 15:06:36,043 — models.lenet — INFO — Training completed.


In [11]:
lenet_model.evaluate()

TypeError: BaseModel.evaluate() missing 1 required positional argument: 'test_data'

### MiniVGGNet Train

In [20]:
mini_vgg_net_model = mini_vgg_net.MiniVGGNet(params_yaml=PARAMS_YAML)
mini_vgg_net_model.build()
mini_vgg_net_model.get_summary()

2024-09-28 14:58:24,365 — utils.utils — INFO — yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully
2024-09-28 14:58:24,422 — models.mini_vgg_net — INFO — model is built


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 200704)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │   102,760,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,829,857 (392.26 MB)

 Trainable params: 102,828,449 (392.26 MB)

 Non-trainable params: 1,408 (5.50 KB)

### ShallowNet Train

In [ ]:
# shallownet_model = shallownet.ShallowNet()